In [1]:
# !unrar x "/content/drive/MyDrive/Colab Notebooks/Dataset/IP02/classification/train2.rar" "/content/drive/MyDrive/Colab Notebooks/Dataset/IP02/classification/train/"

In [2]:
# while True:
#   pass

In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.mobilenet import MobileNet ,preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
from keras.models import Sequential
from keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import os
import shutil
import cv2 as cv
from keras.callbacks import EarlyStopping ,ReduceLROnPlateau , ModelCheckpoint

In [4]:
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/Dataset/IP02/classification/train"
No_of_files = len(os.listdir(train_data_dir))
print(No_of_files)

102


In [5]:
# from google.colab.patches import cv2_imshow
# path = '/content/drive/MyDrive/Colab Notebooks/Dataset/IP04/JPEGImages/IP001000262.jpg'
# img = cv.imread(path)
# cv2_imshow(img)

##Gobal Constant

##Model Define

In [6]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/IP02/classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/IP02/classification/val'
test_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset/IP02/classification/test'
IMG_SIZE = (224,224)

training=tf.keras.preprocessing.image.ImageDataGenerator(
    zca_epsilon=1e-06,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.05
).flow_from_directory(train_dir,batch_size=32,target_size=(256,256),subset="training")

validing=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.05
).flow_from_directory(train_dir,batch_size=16,target_size=(224,224),subset='validation',shuffle=True)

testing=tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
).flow_from_directory(test_dir,batch_size=16,target_size=(224,224),shuffle=True)

Found 42888 images belonging to 102 classes.
Found 2207 images belonging to 102 classes.
Found 22619 images belonging to 102 classes.


## Train Model

In [7]:
pretrain_model=keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(256,256,3))

94781440/94765736 [==============================] - 0s 0us/step


In [8]:
optimizer=Adam(learning_rate=0.01,beta_1=0.9,beta_2=0.99)

In [9]:
EarlyStop=EarlyStopping(patience=10,restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_accuracy',verbose=2,factor=0.5,min_lr=0.00001)
# callback=[EarlyStop , Reduce_LR]
model_check=ModelCheckpoint(filepath='/content/model_resnet.hdf5',monitor='val_loss',verbose=1,save_best_only=True, mode='auto')
callback=[EarlyStop , Reduce_LR, model_check]

In [10]:
pretrain_model.trainable=False

In [11]:
model=Sequential([
    pretrain_model,
    MaxPooling2D(3,2),
    Flatten(),
    Dense(128,activation='relu'),
    BatchNormalization(),
    Dense(1024,activation='relu'),
    BatchNormalization(),
    Dense(512,activation='relu'),
    BatchNormalization(),
    Dense(102,activation='softmax')
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 2048)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 128)               2359424   
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 1024)              1

In [13]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=["accuracy"])

In [14]:
history=model.fit(training,validation_data=validing,epochs=50,batch_size=32,
                  steps_per_epoch=len(training) // 32 ,validation_steps=len(validing) // 16,
                  callbacks=callback, verbose=1)

Epoch 1/50
41/41 [==============================] - ETA: 0s - loss: 5.2822 - accuracy: 0.1113
Epoch 00001: val_loss improved from inf to 38.16346, saving model to /content/model_resnet.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


41/41 [==============================] - 341s 8s/step - loss: 5.2822 - accuracy: 0.1113 - val_loss: 38.1635 - val_accuracy: 0.0078 - lr: 0.0100
Epoch 2/50
41/41 [==============================] - ETA: 0s - loss: 4.0354 - accuracy: 0.2020
Epoch 00002: val_loss improved from 38.16346 to 6.41654, saving model to /content/model_resnet.hdf5
41/41 [==============================] - 317s 8s/step - loss: 4.0354 - accuracy: 0.2020 - val_loss: 6.4165 - val_accuracy: 0.1484 - lr: 0.0100
Epoch 3/50
41/41 [==============================] - ETA: 0s - loss: 3.7062 - accuracy: 0.2142
Epoch 00003: val_loss improved from 6.41654 to 4.58001, saving model to /content/model_resnet.hdf5
41/41 [==============================] - 313s 8s/step - loss: 3.7062 - accuracy: 0.2142 - val_loss: 4.5800 - val_accuracy: 0.2031 - lr: 0.0100
Epoch 4/50
41/41 [==============================] - ETA: 0s - loss: 3.3833 - accuracy: 0.2508
Epoch 00004: val_loss improved from 4.58001 to 3.55276, saving model to /content/model_re